<a href="https://colab.research.google.com/github/ShyamRavidath/colorectal_cancer_classifier-using-deeplearning/blob/main/deep_colorectal_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Path to the dataset folder in Google Drive
dataset_path = '/content/drive/MyDrive/CRC-VAL-HE-7K/CRC-VAL-HE-7K'

# Define classes
benign_classes = ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM']
malignant_classes = ['STR', 'TUM']
all_classes = benign_classes + malignant_classes

# Load images and labels
images = []
labels = []

for idx, class_name in enumerate(all_classes):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.exists(class_path):
        print(f"Warning: {class_path} does not exist. Skipping.")
        continue
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Unable to read image {img_path}. Skipping.")
            continue
        img = cv2.resize(img, (128, 128))  # Resize images to a fixed size
        images.append(img)
        labels.append(idx)  # Assign label based on class index

# Check if images were loaded
if len(images) == 0:
    raise ValueError("No images were loaded. Check the dataset path and structure.")
else:
    print(f"Loaded {len(images)} images.")

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize images
images = images / 255.0

# Convert labels to one-hot encoding
num_classes = len(all_classes)
labels = to_categorical(labels, num_classes)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

Loaded 7194 images.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 9)                   │           1,161 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,801 (12.61 MB)

 Trainable params: 3,305,801 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.4093 - loss: 1.5905 - val_accuracy: 0.5705 - val_loss: 1.1610
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.6844 - loss: 0.9118 - val_accuracy: 0.7964 - val_loss: 0.5712
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.7641 - loss: 0.6713 - val_accuracy: 0.8172 - val_loss: 0.4872
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.7846 - loss: 0.6238 - val_accuracy: 0.8568 - val_loss: 0.4272
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.8275 - loss: 0.5188 - val_accuracy: 0.8499 - val_loss: 0.4350
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.8323 - loss: 0.4704 - val_accuracy: 0.8596 - val_loss: 0.3768
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.8582 - loss: 0.4031 - val_accuracy: 0.8304 - val_loss: 0.5219
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.8711 - loss: 0.3598 - val_accu

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Plot training history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'X_test' is not defined

In [ ]:
model.save('/content/drive/MyDrive/Model.h5')


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/drive/MyDrive/Model.h5')

# Recompile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Verify the model is compiled
print("Model recompiled successfully.")

model.save('/content/drive/MyDrive/Model.h5', save_format='tf')


Model recompiled successfully.


In [ ]:
import cv2
import numpy as np
from google.colab import files
from tensorflow.keras.models import load_model

# Load the model
model = load_model('/content/drive/MyDrive/Model.h5')


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Verify the model is compiled
print("Model recompiled successfully.")

model.save('/content/drive/MyDrive/Model.h5', save_format='tf')

benign_classes = ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM']
malignant_classes = ['STR', 'TUM']
all_classes = benign_classes + malignant_classes

def predict_image(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Failed to load image from {image_path}. Check the path or file format.")

    # Resize and preprocess the image
    img = cv2.resize(img, (128, 128))  # Use the same size as during training
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Predict
    prediction = model.predict(img)
    predicted_class_idx = np.argmax(prediction)
    predicted_class = all_classes[predicted_class_idx]
    confidence_score = prediction[0][predicted_class_idx]
    return predicted_class, confidence_score

# Example usage
try:
    uploaded = files.upload()
    for filename, content in uploaded.items():
      image_path = '/content/'+filename
      predicted_class, confidence_score = predict_image(image_path)
      class_img = "Benign" if predicted_class in benign_classes else "Malignant"
      print(f"Tissue is {class_img} \n Predicted Class: {predicted_class}, Confidence Score: {confidence_score:.4f}")

except Exception as e:
    print(e)


Model recompiled successfully.


Saving LYM-TCGA-AAWGSCHH.tif to LYM-TCGA-AAWGSCHH.tif
Saving LYM-TCGA-AFIDYMYA.tif to LYM-TCGA-AFIDYMYA.tif
Saving LYM-TCGA-AFILHSCE.tif to LYM-TCGA-AFILHSCE.tif
Saving LYM-TCGA-AGDQANVL.tif to LYM-TCGA-AGDQANVL.tif
Saving LYM-TCGA-AHVYIAAM.tif to LYM-TCGA-AHVYIAAM.tif
Saving LYM-TCGA-AIVQRVDG.tif to LYM-TCGA-AIVQRVDG.tif
Saving LYM-TCGA-AKGHMWKD.tif to LYM-TCGA-AKGHMWKD.tif
Saving LYM-TCGA-AKTERMWV.tif to LYM-TCGA-AKTERMWV.tif
Saving LYM-TCGA-ALLEFAMW.tif to LYM-TCGA-ALLEFAMW.tif
Saving LYM-TCGA-ALMSHWVW.tif to LYM-TCGA-ALMSHWVW.tif
Saving LYM-TCGA-AMDGAIIS.tif to LYM-TCGA-AMDGAIIS.tif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Tissue is Benign 
 Predicted Class: LYM, Confidence Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Tissue is Benign 
 Predicted Class: LYM, Confidence Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Tissue is Benign 
 Predicted Class: LYM, Confidence Score: 1.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Tissue is Benign 
 Predicted Class: LYM, Confidence Sco